In [ ]:
# 📌 Section 1: Install dependencies (if not already installed)
!pip install transformers sentencepiece nltk --quiet

In [ ]:
# 📌 Section 2: Import libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 📌 Section 3: Load T5 Model and Tokenizer
model_name = "t5-small"  # You can also try "t5-base" for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# 📌 Section 4: Define Summarization Function
def summarize_text_t5(text, max_input_length=512, max_output_length=150):
    # Preprocess input text
    input_text = "summarize: " + text.strip().replace("\n", " ")
    input_ids = tokenizer.encode(
        input_text,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True
    )

    # Generate summary
    summary_ids = model.generate(
        input_ids,
        max_length=max_output_length,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
# 📌 Section 5: Sample Text (You can replace this with text from a PDF)
sample_text = """
Recent advancements in artificial intelligence and deep learning have enabled the development
of sophisticated text summarization systems. These systems aim to condense long documents into
short, informative summaries that retain the core meaning of the original content. Extractive
approaches select key sentences, while abstractive models like T5 generate new sentences using
language understanding. In this paper, we review current techniques and propose a hybrid
framework that leverages the advantages of both.
"""

In [ ]:
# 📌 Section 6: Generate Summary
summary = summarize_text_t5(sample_text)
print("Original Text:\n", sample_text)
print("\nGenerated Summary:\n", summary)

Original Text:
 
Recent advancements in artificial intelligence and deep learning have enabled the development 
of sophisticated text summarization systems. These systems aim to condense long documents into 
short, informative summaries that retain the core meaning of the original content. Extractive 
approaches select key sentences, while abstractive models like T5 generate new sentences using 
language understanding. In this paper, we review current techniques and propose a hybrid 
framework that leverages the advantages of both.


Generated Summary:
 recent advances in artificial intelligence and deep learning have enabled the development of sophisticated text summarization systems. these systems aim to condense long documents into short, informative summaries that retain the core meaning of the original content. extractive approaches select key sentences, while abstractive models like T5 generate new sentences using language understanding.


In [ ]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def extractive_textrank_summary(text, num_sentences=2):
    sentences = sent_tokenize(text)
    tfidf = TfidfVectorizer().fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf)

    scores = similarity_matrix.sum(axis=1)
    ranked_sentences = [sentences[i] for i in scores.argsort()[-num_sentences:][::-1]]

    return " ".join(ranked_sentences)

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Compare
extractive_summary = extractive_textrank_summary(sample_text)
print("\nTextRank Summary:\n", extractive_summary)


TextRank Summary:
 In this paper, we review current techniques and propose a hybrid 
framework that leverages the advantages of both. 
Recent advancements in artificial intelligence and deep learning have enabled the development 
of sophisticated text summarization systems.
